# Loan Prediction Analysis

## Loading the datasets

In [1]:
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.impute import KNNImputer
import plotly.express as px

In [2]:
dat=pd.read_csv('./train.csv')

# Basic Attributes of the Dataset

In [3]:
dat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 614 entries, 0 to 613
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Loan_ID            614 non-null    object 
 1   Gender             601 non-null    object 
 2   Married            611 non-null    object 
 3   Dependents         599 non-null    object 
 4   Education          614 non-null    object 
 5   Self_Employed      582 non-null    object 
 6   ApplicantIncome    614 non-null    int64  
 7   CoapplicantIncome  614 non-null    float64
 8   LoanAmount         592 non-null    float64
 9   Loan_Amount_Term   600 non-null    float64
 10  Credit_History     564 non-null    float64
 11  Property_Area      614 non-null    object 
 12  Loan_Status        614 non-null    object 
dtypes: float64(4), int64(1), object(8)
memory usage: 62.5+ KB


In [4]:
dat.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History
count,614.000000,614.000000,592.000000,600.00000,564.000000
mean,5403.459283,1621.245798,146.412162,342.00000,0.842199
std,6109.041673,2926.248369,85.587325,65.12041,0.364878
min,150.000000,0.000000,9.000000,12.00000,0.000000
25%,2877.500000,0.000000,100.000000,360.00000,1.000000
50%,3812.500000,1188.500000,128.000000,360.00000,1.000000
75%,5795.000000,2297.250000,168.000000,360.00000,1.000000
max,81000.000000,41667.000000,700.000000,480.00000,1.000000


In [5]:
dat.shape


(614, 13)

In [6]:
dat.isnull().sum()

Loan_ID               0
Gender               13
Married               3
Dependents           15
Education             0
Self_Employed        32
ApplicantIncome       0
CoapplicantIncome     0
LoanAmount           22
Loan_Amount_Term     14
Credit_History       50
Property_Area         0
Loan_Status           0
dtype: int64

#### Here we can see that accept Loan_ID,Education,Both Incomes, Property area and loan status all of them have some sort of null value either numerical or categoriacal.

## For numerical data using KNN imputer

In [7]:
imputer=KNNImputer()
dat[['LoanAmount','CoapplicantIncome','ApplicantIncome','Loan_Amount_Term','Credit_History']]=imputer.fit_transform(dat[['LoanAmount','CoapplicantIncome','ApplicantIncome','Loan_Amount_Term','Credit_History']])

## For Categorical attributes
### Firstly Gender and Married don't play any role is loan getting approved so removing them, then for the remaining two categorical variables imputing from the mode.

In [8]:
dat=dat.drop(['Gender','Married'],axis=1)

In [9]:
dat['Self_Employed']=dat['Self_Employed'].fillna(dat['Self_Employed'].mode()[0])
dat['Dependents']=dat['Dependents'].fillna(dat['Dependents'].mode()[0])

## Dropping loan Id

In [10]:
dat=dat.drop(['Loan_ID'],axis=1)

## Aa both Applicant Income and Co-applicant income are both incomes so adding them to form a new column Total Income.

In [11]:
df=dat.copy()
df['TotalIncome']=df['ApplicantIncome']+df['CoapplicantIncome']

In [12]:
df=df.drop(['ApplicantIncome','CoapplicantIncome'],axis=1)

In [13]:
df['LoanAmount']=np.log(df['LoanAmount'])
df['TotalIncome']=np.log(df['TotalIncome'])

In [14]:
df.head()

,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome
0,0,Graduate,No,4.995860,360.0,1.0,Urban,Y,8.674026
1,1,Graduate,No,4.852030,360.0,1.0,Rural,N,8.714568
2,0,Graduate,Yes,4.189655,360.0,1.0,Urban,Y,8.006368
3,0,Not Graduate,No,4.787492,360.0,1.0,Urban,Y,8.505323
4,0,Graduate,No,4.948760,360.0,1.0,Urban,Y,8.699515


## Label Encoding using sklearn

In [15]:
from sklearn.preprocessing import LabelEncoder

In [16]:
le=LabelEncoder()
df['Dependents']=le.fit_transform(df['Dependents'])
df['Education']=le.fit_transform(df['Education'])
df['Self_Employed']=le.fit_transform(df['Self_Employed'])
df['Property_Area']=le.fit_transform(df['Property_Area'])

In [17]:
df.head()

,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,TotalIncome
0,0,0,0,4.995860,360.0,1.0,2,Y,8.674026
1,1,0,0,4.852030,360.0,1.0,0,N,8.714568
2,0,0,1,4.189655,360.0,1.0,2,Y,8.006368
3,0,1,0,4.787492,360.0,1.0,2,Y,8.505323
4,0,0,0,4.948760,360.0,1.0,2,Y,8.699515


## Model Preparation

In [18]:
x=df.drop('Loan_Status',axis=1)
cl=df.Loan_Status

## Importing Naive Bayes Classifier

In [19]:
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import train_test_split as tts
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold

In [20]:
k=10
kf=KFold(n_splits=k,random_state=None)
model=GaussianNB()
for train_index , test_index in kf.split(x):
    xtrain,xtest=x.iloc[train_index,:],x.iloc[test_index,:]
    ytrain,ytest=cl[train_index],cl[test_index]
model.fit(xtrain,ytrain)

GaussianNB()

In [62]:
model.score(xtest,ytest)*100

86.88524590163934

In [63]:
model.predict(xtest)

array(['N', 'N', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y',
       'Y', 'N', 'Y', 'N', 'Y', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'N', 'N', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'N', 'Y', 'Y', 'N', 'Y', 'Y', 'Y', 'Y',
       'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'Y', 'N'], dtype='<U1')

In [64]:
xtest.head()

,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,TotalIncome
553,0,1,0,5.198497,360.0,0.0,2,8.473659
554,0,0,0,4.882802,180.0,0.0,0,8.969415
555,1,0,0,3.258097,360.0,1.0,1,8.779557
556,0,0,0,4.430817,360.0,1.0,2,8.364508
557,3,0,1,5.560682,360.0,1.0,1,9.224145


## Further code to determine the maximum loan that can be approved in case of disapproval of the Loan Request.

In [65]:
# Data with 'No' as loan status
dataa=[[1,0,0,13.258097,360,1,1,8.364508]]
daf = pd.DataFrame(dataa, columns = ['Dependents','Education','Self_Employed','LoanAmount','Loan_Amount_Term','Credit_History','Property_Area','TotalIncome'])

In [66]:
daf

,Dependents,Education,Self_Employed,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,TotalIncome
0,1,0,0,13.258097,360,1,1,8.364508


In [67]:
model.predict(daf)[0]

'N'

In [68]:
if model.predict(daf)[0]=='Y':
    print("Loan Approved")
else:
    for i in np.arange(daf.LoanAmount[0],0,-.001):
        daf.LoanAmount=i
        if model.predict(daf)[0]=='Y':
            print("Maximum Amount that can be approved:",(daf.LoanAmount[0]))
            break

Maximum Amount that can be approved: 9.067097000002322
